In [2]:
import pandas as pd

In [121]:
df = pd.read_csv("data/filtered/Top1PercentJobWanted.csv")

## 2.2 数据预处理与清洗

### 2.2.1 数据划分

#### 地域数据划分

基于原始数据，本作品依据行政区划首字母将地域信息划分为一级行政区划和二级行政区划。即认为属性“city”中所有首字母相同的二级行政区划属于同一类一级行政区划。本作品认为各省份内部经济状况相似，不同地区间的经济差异直接影响薪酬水平。


In [122]:
df['province'] = df['city'].astype(str).str[0]

#### 薪资数据划分

原始数据包含年薪、月薪、周薪等其中不同薪酬表达形式。为了使分析更加直观，本作品依据《中华人民共和国劳动合同法》将薪酬数据划分为固定用工和灵活用工两类。

为统一薪资的表示模式，本作品通过正则表达式匹配抽取每条数据的薪资上界和下界，同时依据劳动和社会保障部颁布的《关于职工全年月平均工作时间和工资折算问题的通知》，将不同薪酬模式换算为年薪。对于单结及面议的薪酬信息，由于其不确定性高且数据量少，故按同职位其他薪酬的平均数处理。

In [123]:
import re

# 初始化，这个-2其实没有什么特别的意义
df['salary_min'] = df['salary_max'] = -2 
df['salary_base'] = -2
df['salary_type'] = ''

for index, row in df.iterrows():
    if row['salary'] == '面议':
        df.at[index, 'salary_type'] = 'N'
        continue
    
    if row['salary'].endswith('单'):
        df.at[index, 'salary_type'] = 'U'
        continue
            
    if re.match('(.+)-(.+)K·(.+)薪', row['salary']):
        match = re.match('(.+)-(.+)K·(.+)薪', row['salary'])
        df.at[index, 'salary_type'] = 'B'
        df.at[index, 'salary_min'] = int(match[1]) * int(match[3]) * 1000
        df.at[index, 'salary_max'] = int(match[2]) * int(match[3]) * 1000
        df.at[index, 'salary_base'] = int(match[3])
        continue
        
    if re.match('(.+)-(.+)元/天', row['salary']):
        match = re.match('(.+)-(.+)元/天', row['salary'])
        df.at[index, 'salary_type'] = 'D'
        df.at[index, 'salary_min'] = int(match[1]) * 250 
        df.at[index, 'salary_max'] = int(match[2]) * 250
        continue
    
    if re.match('(.+)-(.+)元/时', row['salary']):
        match = re.match('(.+)-(.+)元/时', row['salary'])
        df.at[index, 'salary_type'] = 'H'
        df.at[index, 'salary_min'] = int(match[1]) * 2000
        df.at[index, 'salary_max'] = int(match[2]) * 2000
        continue
            
    if re.match('(.+)-(.+)元/周', row['salary']):
        match = re.match('(.+)-(.+)元/周', row['salary'])
        df.at[index, 'salary_type'] = 'W'
        df.at[index, 'salary_min'] = int(match[1]) * 50
        df.at[index, 'salary_max'] = int(match[2]) * 50
        continue
    
    if re.match('(.+)-(.+)元/月', row['salary']):
        match = re.match('(.+)-(.+)元/月', row['salary'])
        df.at[index, 'salary_type'] = 'M'
        df.at[index, 'salary_min'] = int(match[1]) * 12
        df.at[index, 'salary_max'] = int(match[2]) * 12
        df.at[index, 'salary_base'] = 12
        continue
    
    if re.match('(.+)-(.+)K', row['salary']):
        match = re.match('(.+)-(.+)K', row['salary'])
        df.at[index, 'salary_type'] = 'M'
        df.at[index, 'salary_min'] = int(float(match[1]) * 1000 * 12)
        df.at[index, 'salary_max'] = int(float(match[2]) * 1000 * 12)
        df.at[index, 'salary_base'] = 12
        continue
    
    if re.match('(.+)-(.+)', row['salary']):
        match = re.match('(.+)-(.+)', row['salary'])
        df.at[index, 'salary_type'] = 'M'
        df.at[index, 'salary_min'] = int(match[1]) * 12
        df.at[index, 'salary_max'] = int(match[2]) * 12
        df.at[index, 'salary_base'] = 12
        continue

    # 唯一的一个特殊处理的数据"6K以上"
    df.at[index, 'salary_type'] = 'M'
    df.at[index, 'salary_min'] = 72000
    df.at[index, 'salary_max'] = 72000
    df.at[index, 'salary_base'] = 12

### 2.2.2 数据清洗

#### 固定用工

考虑到灵活用工和固定用工的内在差异，本作品基于统一的薪酬模式，对两类形式的 招聘信息分别进行数据清洗。首先基于一级行政区划与职位将招聘信息分为若干小组，对于分组内的招聘信息，本作品采取四分位距(IQR)方法检测异常值并排除。

In [132]:
fixed_df = df[(df['salary_type'] == 'M') | (df['salary_type'] == 'B')]
fixed_df

,JobTitle,city,salary,experience,education,company,companyType,province,salary_min,salary_max,salary_base,salary_type
0,bacd30a8f578d3c3f4e75bd227af5fa5qw,K445,9-14K,Eqh,Gx,company_598986,type_CbBLwi,K,108000,168000,12,M
1,ce08ca90188a7b2f206decc2287a2bdeQy,K445,11-15K,EzN,Gx,company_045715,type_NKZQoO,K,132000,180000,12,M
2,f4d163a8c601f529869974551adfefc8PY,K445,6-7K,Eas,GI,company_212434,type_fFlUqG,K,72000,84000,12,M
3,2ba830252c837dc1d5843cdf5e806b60Gb,K445,4-5K,EdD,GI,company_914209,type_vjgYNw,K,48000,60000,12,M
4,65e915067ec7ca2082c1bfc73ffbc06bhv,K445,5-9K,Eas,GI,company_940634,type_PanQpo,K,60000,108000,12,M
...,...,...,...,...,...,...,...,...,...,...,...,...
192477,d669436e131961c0873228a725197ed5eg,V449,4-8K,Eas,GI,company_723487,type_yZblEo,V,48000,96000,12,M
192478,bdbba0cecd9ff8e1b59e8dbf90faf9ebkU,V449,10-15K,Eas,GI,company_131298,type_lOdYUb,V,120000,180000,12,M
192479,bc06e4cf4937d73282ccc36f22daa029yi,V449,5-10K,EdD,GI,company_008885,type_lOdYUb,V,60000,120000,12,M
192480,7d3783d923cab44fde9a8d0fe8aa7570DN,V449,8-13K,Eqh,Gx,company_926342,type_lYwbWD,V,96000,156000,12,M


对固定用工的数据，先按照'JobTitle'和'province'分组，在组内采用IQR的方式检测异常值，如果薪资上下界有离群值的话就认为数据异常

In [125]:
fixed_grouped = fixed_df.groupby(['JobTitle', 'province'])

In [126]:
def IQRfilter(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    L = Q1 - 1.5 * IQR
    U = Q3 + 1.5 * IQR
    return L, U

In [127]:
for name, group in fixed_grouped:
    L_min, U_min = IQRfilter(group['salary_min'])
    L_max, U_max = IQRfilter(group['salary_max'])
    for index, row in group.iterrows():
        salary_min = row['salary_min']
        salary_max = row['salary_max']
        if (salary_min <= L_min or salary_min >= U_min) or (salary_max <= L_max or salary_max >= U_max):
            fixed_df.drop(index=index, inplace=True)
            df.drop(index=index, inplace=True)

161921

考虑到IQR在数据量较小的组上的误判现象较明显，对数据量 < 10的组进行人工处理，最终保留 163786 条固定用工数据

#### 灵活用工

In [135]:
flex_df = df[(df['salary_type'] != 'M') & (df['salary_type'] != 'B')].drop(columns='salary_base')
flex_df

,JobTitle,city,salary,experience,education,company,companyType,province,salary_min,salary_max,salary_type
47,f4d163a8c601f529869974551adfefc8PY,K445,5-20元/时,EdD,Gx,company_611057,type_kiIJXj,K,10000,40000,H
52,14048d07f6236aea0f8b7b390d680e7blh,K445,130-160元/天,Eby,GP,company_107722,type_EguSJg,K,32500,40000,D
88,534153744af8afe2c702dac2f551bb52kB,K445,50-100元/天,Eby,GP,company_246114,type_fWnADK,K,12500,25000,D
114,14048d07f6236aea0f8b7b390d680e7blh,K115,290-300元/天,Eby,GP,company_348714,type_CbBLwi,K,72500,75000,D
117,14048d07f6236aea0f8b7b390d680e7blh,K115,180-200元/天,Eby,GP,company_564821,type_BLfSmG,K,45000,50000,D
...,...,...,...,...,...,...,...,...,...,...,...
191942,76b78c5d7d8ff1f1be59bba95db2adbbBV,V449,60-90元/天,Eas,GI,company_040860,type_cPyqFN,V,15000,22500,D
192001,422e306ecc84c742bd1aab21cd80edd2vv,V449,50-80元/天,Eby,GP,company_703016,type_yZblEo,V,12500,20000,D
192165,9f8c93e79b3b6b937ba53304f4c66c56NZ,V449,120-150元/时,EdD,Gx,company_241215,type_lOdYUb,V,240000,300000,H
192174,724979c7aca0f203024e5cfc80875885sO,V449,100-150元/天,Eby,GP,company_428450,type_CbBLwi,V,25000,37500,D


根据人力资源社会保障部官网公布的《全国各省、自治区、直辖市最低工资标准情况》<https://baijiahao.baidu.com/s?id=1803897153991634500&wfr=spider&for=pc>筛除salary_min < 12500的数据

In [136]:
for index, row in flex_df.iterrows():
    if row['salary_min'] < 12500:
        flex_df.drop(index=index, inplace=True)
        df.drop(index=index, inplace=True)

,JobTitle,city,salary,experience,education,company,companyType,province,salary_min,salary_max,salary_type
52,14048d07f6236aea0f8b7b390d680e7blh,K445,130-160元/天,Eby,GP,company_107722,type_EguSJg,K,32500,40000,D
88,534153744af8afe2c702dac2f551bb52kB,K445,50-100元/天,Eby,GP,company_246114,type_fWnADK,K,12500,25000,D
114,14048d07f6236aea0f8b7b390d680e7blh,K115,290-300元/天,Eby,GP,company_348714,type_CbBLwi,K,72500,75000,D
117,14048d07f6236aea0f8b7b390d680e7blh,K115,180-200元/天,Eby,GP,company_564821,type_BLfSmG,K,45000,50000,D
121,769ab85fa91937cb72301dbee9a364f7NA,K115,250-300元/天,Eby,Gw,company_916992,type_ZnqLAd,K,62500,75000,D
...,...,...,...,...,...,...,...,...,...,...,...
191942,76b78c5d7d8ff1f1be59bba95db2adbbBV,V449,60-90元/天,Eas,GI,company_040860,type_cPyqFN,V,15000,22500,D
192001,422e306ecc84c742bd1aab21cd80edd2vv,V449,50-80元/天,Eby,GP,company_703016,type_yZblEo,V,12500,20000,D
192165,9f8c93e79b3b6b937ba53304f4c66c56NZ,V449,120-150元/时,EdD,Gx,company_241215,type_lOdYUb,V,240000,300000,H
192174,724979c7aca0f203024e5cfc80875885sO,V449,100-150元/天,Eby,GP,company_428450,type_CbBLwi,V,25000,37500,D


进一步人工处理相近条件下薪资差异过大的少量异常数据

## 2.3 最终数据构建与特征

In [3]:
salary_df = pd.read_csv('data/final/SalaryFinal.csv')
salary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166118 entries, 0 to 166117
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   _id          166118 non-null  object
 1   city         166118 non-null  object
 2   salary       166118 non-null  object
 3   experience   166118 non-null  object
 4   education    166118 non-null  object
 5   company      166118 non-null  object
 6   companyType  166118 non-null  object
 7   salary_type  166118 non-null  object
 8   salary_min   166118 non-null  int64 
 9   salary_max   166118 non-null  int64 
 10  province     166118 non-null  object
 11  is_outlier   166118 non-null  bool  
dtypes: bool(1), int64(2), object(9)
memory usage: 14.1+ MB


数据包含 166,118 条招聘记录，涉及 1,713 个独立岗位。数据覆盖 26 个一级行政区划、370 个二级行政区划，136,149 家企业，158 个不同的行业类别，包含 8 种不同的工作经验要求和 8 种学历要求类别。